Let's first define classifiers and some functions that we will need later.

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
import sklearn

print(sklearn.__version__)

# Disable warnings for some sklearn classifiers
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Define classifiers
classifiers = {
    "knn, k = 1": KNeighborsClassifier(1),
    "knn, k = 1, weights = distance": KNeighborsClassifier(1, weights='distance'),
    "knn, k = 3": KNeighborsClassifier(3),
    "knn, k = 3, weights = distance": KNeighborsClassifier(3, weights='distance'),
    "knn, k = 5": KNeighborsClassifier(5),
    "knn, k = 5, weights = distance": KNeighborsClassifier(5, weights='distance'),
    "knn, k = 7": KNeighborsClassifier(7),
    "knn, k = 7, weights = distance": KNeighborsClassifier(7, weights='distance'),
    "decision tree, gini": DecisionTreeClassifier(criterion='gini'),
    "decision tree, entropy": DecisionTreeClassifier(criterion='entropy'),
    "random forest, gini": RandomForestClassifier(n_estimators=100, criterion='gini'),
    "random forest, entropy": RandomForestClassifier(n_estimators=100, criterion='entropy'),
    "naive bayes": GaussianNB(),
    "support vector machine, kernel=linear, c=0.025": SVC(kernel="linear", C=0.025),
    "support vector machine, kernel=linear, c=0.05": SVC(kernel="linear", C=0.05),
    "support vector machine, kernel=linear, c=0.1": SVC(kernel="linear", C=0.1),
    "support vector machine, kernel=linear, c=0.2": SVC(kernel="linear", C=0.2),
    "support vector machine, kernel=rbf, c=0.025": SVC(kernel='rbf', C=0.1)
}

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))

def evaluate_classifiers(x_train, x_test, y_train, y_test, n_iterations=1):
    for classifier_name in classifiers:
        scores = []
        for i in range(n_iterations):
            classifier = classifiers[classifier_name]
            classifier.fit(x_train, y_train)
            test_score = classifier.score(x_test, y_test)
            scores.append(test_score)
        print("{}: {:.3f}".format(classifier_name, np.average(scores)))

def prepare_train_test_set(train_file, test_file=None, normalize=True, merge_datasets=False):
    data_train = pd.read_csv(train_file)
    
    if test_file is None:
        x_train = data_train.iloc[:, :-1].values
        y_train = data_train.iloc[:, 128].values

        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)

    else:
        data_test = pd.read_csv(test_file)

        if merge_datasets:
            data_train = data_train.append(data_test)
            
            x_train = data_train.iloc[:, :-1].values
            y_train = data_train.iloc[:, 128].values

            from sklearn.model_selection import train_test_split
            x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)
        
        else:
            x_train = data_train.iloc[:, :-1].values
            y_train = data_train.iloc[:, 128].values
            x_test = data_test.iloc[:, :-1].values
            y_test = data_test.iloc[:, 128].values

    # Normalize the feautures
    if normalize:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
    
    return x_train, x_test, y_train, y_test

0.20.4


## Evaluating classifiers
Let's first train and test our models on images taken in gazebo simulation.

We can see that most of the classifiers perform extremely well. The only exception being decision tree, but it still works fairly well.

There are a few reasons why all the classifiers work really well.

1. The `face_recognition` library does a very good job encoding the faces into embeddings. As we can see in an example later in this notebook, when using only the original images of the faces as train set and faces from simulation as test set, the classifiers still perform very well.

2. We only need to recognize 21 faces, which makes this problem much easier.

3. The nature of the simulation makes this task easier as well. There is no blur in the images recorded in the simulation and the lighting conditions are very simple.

We can notice that KNeighbors algorithm works very well, despite our high dimensional data.

In [10]:
x_train, x_test, y_train, y_test = prepare_train_test_set('../encodings/faces_all_simulation.txt')

evaluate_classifiers(x_train, x_test, y_train, y_test, 20)

knn, k = 7, weights = distance: 0.995
support vector machine, kernel=linear, c=0.05: 1.000
decision tree, entropy: 0.881
random forest, gini: 0.994
random forest, entropy: 0.998
support vector machine, kernel=rbf, c=0.025: 0.990
knn, k = 1, weights = distance: 0.995
naive bayes: 0.995
decision tree, gini: 0.862
support vector machine, kernel=linear, c=0.1: 1.000
support vector machine, kernel=linear, c=0.2: 1.000
knn, k = 3, weights = distance: 0.995
support vector machine, kernel=linear, c=0.025: 1.000
knn, k = 5, weights = distance: 0.995
knn, k = 1: 0.995
knn, k = 3: 0.995
knn, k = 5: 0.995
knn, k = 7: 0.995


We can see that training the models on images from the simulation and the evaluating them on real images yields us very bad results.
This is probably due to the fact that the conditions in which the images were taken are very different.

In [11]:
x_train, x_test, y_train, y_test = prepare_train_test_set('../encodings/faces_all_simulation.txt', '../encodings/faces_all_camera.txt')

evaluate_classifiers(x_train, x_test, y_train, y_test, 20)

knn, k = 7, weights = distance: 0.422
support vector machine, kernel=linear, c=0.05: 0.422
decision tree, entropy: 0.325
random forest, gini: 0.421
random forest, entropy: 0.419
support vector machine, kernel=rbf, c=0.025: 0.417
knn, k = 1, weights = distance: 0.426
naive bayes: 0.417
decision tree, gini: 0.301
support vector machine, kernel=linear, c=0.1: 0.422
support vector machine, kernel=linear, c=0.2: 0.422
knn, k = 3, weights = distance: 0.426
support vector machine, kernel=linear, c=0.025: 0.422
knn, k = 5, weights = distance: 0.422
knn, k = 1: 0.426
knn, k = 3: 0.426
knn, k = 5: 0.422
knn, k = 7: 0.417


We can observe that training the models on images taken on a physical camera gives us very good results as well. This means that our pipeline could be used for face recognition on real robots as well.

In [12]:
x_train, x_test, y_train, y_test = prepare_train_test_set('../encodings/faces_all_camera.txt')

evaluate_classifiers(x_train, x_test, y_train, y_test, 20)

knn, k = 7, weights = distance: 0.956
support vector machine, kernel=linear, c=0.05: 0.978
decision tree, entropy: 0.663
random forest, gini: 0.946
random forest, entropy: 0.936
support vector machine, kernel=rbf, c=0.025: 0.244
knn, k = 1, weights = distance: 0.956
naive bayes: 0.933
decision tree, gini: 0.778
support vector machine, kernel=linear, c=0.1: 0.978
support vector machine, kernel=linear, c=0.2: 0.978
knn, k = 3, weights = distance: 0.956
support vector machine, kernel=linear, c=0.025: 0.978
knn, k = 5, weights = distance: 0.956
knn, k = 1: 0.956
knn, k = 3: 0.978
knn, k = 5: 0.978
knn, k = 7: 0.956


Mixing the two datasets (images taken in ros and images taken with a physical camera) yields very good results as well. They are not as good as with training the models on the two datasets seperately. We see no value in this approach because real world and simulation are two very different environments and it would be better to create a two classifiers, one for each environment than one classifier for both occasions. We could however in this way teach the model to recognize faces in different lighting conditions, but this is easier done with manipulating images taken in the simulation, for example darkening them.

In [13]:
x_train, x_test, y_train, y_test = prepare_train_test_set('../encodings/faces_all_simulation.txt', '../encodings/faces_all_camera.txt', merge_datasets=True)

evaluate_classifiers(x_train, x_test, y_train, y_test, 20)

knn, k = 7, weights = distance: 0.949
support vector machine, kernel=linear, c=0.05: 0.970
decision tree, entropy: 0.765
random forest, gini: 0.950
random forest, entropy: 0.949
support vector machine, kernel=rbf, c=0.025: 0.857
knn, k = 1, weights = distance: 0.987
naive bayes: 0.873
decision tree, gini: 0.779
support vector machine, kernel=linear, c=0.1: 0.970
support vector machine, kernel=linear, c=0.2: 0.979
knn, k = 3, weights = distance: 0.979
support vector machine, kernel=linear, c=0.025: 0.962
knn, k = 5, weights = distance: 0.954
knn, k = 1: 0.987
knn, k = 3: 0.979
knn, k = 5: 0.954
knn, k = 7: 0.945


Now let's calculate confusion matrices for a few of the more accurate classifiers:

ImportError: cannot import name plot_confusion_matrix